In [ ]:


class GetTweetsKeyWords:
    """ Get Tweets Specific to a Country and with Certain Keywords"""

    def __init__(
        self,
        bearer_token: str,
        start_date: str,
        end_date: str,
        keywords: str,
        amount: int,
        country: str
    ):
        self.bearer_token = bearer_token
        self.search_url = "https://api.twitter.com/2/tweets/search/all"
        self.start_date = start_date
        self.end_date = end_date
        self.keywords = keywords
        self.amount = amount
        self.country = country

    def create_headers(self, bearer_token):
        headers = {"Authorization": "Bearer {}".format(self.bearer_token)}
        return headers

    def connect_to_endpoint(self, url, headers, params):
        time.sleep(1)
        response = requests.get(
            self.search_url,
            headers=headers,
            params=params
        )
        json_response = response.json()
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return json_response

    def paginate(self, json_response, query_params):
        next_token = json_response["meta"]["next_token"]
        query_params.update({'next_token': next_token})

    def main(self):
        headers = self.create_headers(self.bearer_token)

        query_params = {
            'query': f'(place_country:{self.country}) ({self.keywords})',                        
            'tweet.fields': 'created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld',
            'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified',
            'max_results': 500,
            'expansions': 'attachments.poll_ids,attachments.media_keys,author_id,geo.place_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id',
            'media.fields': 'duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width',
            "start_time": self.start_date,
            "end_time": self.end_date
        }
        json_response = self.connect_to_endpoint(self.search_url, headers, query_params)
        df = json_normalize(json_response['data'])
        df = df.sort_index(axis=1)
        count = 0
        while 'next_token' in json_response['meta'].keys() and count < self.amount:
            time.sleep(1)
            self.paginate(json_response, query_params)
            response = requests.request(
                "GET", self.search_url,
                headers=headers,
                params=query_params
            )
            json_response.update(response.json())
            flat = json_normalize(json_response['data'])
            flat = flat.sort_index(axis=1)
            df = pd.concat([df, flat])
            count = count + 1
        df['date_consulted'] = str(date.today())   

        return df